#**Software gestionale per negozio di prodotti vegani**
Il codice seguente costituisce un software di gestione per un generico negozio di prodotti vegani. Il software è stato realizzato da me, Alessio Feudo, nel contesto del percorso in Data Science di Profesion.AI, modulo di programmazione in Python.
Lo scopo del software è fornire uno strumento in grado di gestire un negozio, contemplando diverse funzionalità.
Il particolare il software è studiato per:


*   Aggiungere prodotti disponibili per la vendita in negozio tramite tastiera.
*   Elencare i prodotti disponibili all'utente.
*   Gestire la vendita dei prodotti all'utente .
*   Immagazzinare le informazioni sulla merce disponibile e su quella venduta   tra diverse esecuzioni del codice stesso.
*   Dare un resoconto economico persistente tra varie esecuzioni sui profitti lordi e netti.

##**Soluzioni messe in atto**
###**Struttura codice**
Per eseguire tali funzionalità si è scelto di fare uso della classe "Market", implementando vari metodi al suo interno per la gestione dei vari compiti. Per tenere memoria dei prodotti presenti in magazzino e delle vendite effettuate durante varie esecuzioni del programma, si è scelto di incamerare tali informazioni all'interno di file csv appositamente generati e poi consultati dal codice stesso, "magazzino.csv" e "vendite.csv". Il codice è quindi composto da due parti: la classe "Market" ed un breve MAIN in cui si istanzia l'oggetto specifico.

###**Gestione input utente**
Tutti gli input richiesti all'utente sono gestiti e controllati dal codice in modo che siano conformi a quanto richiesto, pena l'emissione di messaggi di errore che avvisano l'utente stesso. I vari controlli di input sono progettati per evitare l'interruzione del programma.

###**Fasi di lavoro e casi particolari**
La creazione del software ha attraversato principalemnte due momenti: la fase di scrittura del codice, implementando le varie funzionalità e la fase di validazione del software, costituita da svariati tests atti a provarne l'efficacia e la robustezza.


In particolare, la scrittura della funzione "sell", che gestisce la vendita dei prodotti, si è rivelata complessa, in quanto tante sono le possibilità che deve prevedere. Dopo svariati aggiustamenti, la versione presentata è quella che  gestisce le possibili situazioni al meglio, coniugando intuitività del codice e robustezza.

Tutto il codice è inoltre commentato per permetterne una maggiore comprensione.

In [ ]:
import csv

class Market:
  '''
  This class defines the structure to build an object which will represent a market.
  Attributes:
      - header (list): header to write in files
  '''

  def __init__(self):
    '''
    Constructor of the class.
    '''
    self.header=["PRODOTTO", "QUANTITÀ", "PREZZO", "P-DI-ACQUISTO"]


  def read_file(self,file_name,function_call):
    '''
    The method reads a file. The file could be 'magazzino.csv' or 'vendite.csv'. If the file is not present, an error is raised.
    Args:
        - file_name (str): name of the file to read
        - function_call (str): name of the fuction which calls the current method
    Returns:
        - list: rows of the file
        - None: if file is not present
    '''

    try: #try/except menages file opening with possible errors
        with open(file_name, newline='') as file_csv:
           file_csv.seek(0)  #seek(0) to move the file cursor at the beginning of the file.
           file_reader=csv.reader(file_csv)
           next(file_reader)   #column names are neglected
           rows=list(file_reader) #file rows list
           return rows

    except FileNotFoundError:

        if file_name=="magazzino.csv" and function_call=="add":

             #if the error comes from the file "magazzino.csv" from the method "add_in_market",
             #it means that the market storage is empty and the user is trying to insert goods for the first time.
             #Thus, file is created with the header.

             print("Il file magazzino non è presente. Verrà creato.")
             with open("magazzino.csv", "w+", newline='') as file_csv:
                 file_writer=csv.writer(file_csv)
                 file_writer.writerow(self.header)
             rows=[]    # empty file rows is returned
             return rows

        elif file_name=="magazzino.csv" and function_call=="sell":

             #if error is raised by the method "sell", thus the market storage is empty and nothing can be sold.
             print("File non trovato. Magazzino vuoto.\n")
             return None

        elif file_name=="magazzino.csv" and function_call=="catalogue":
             #if error is raised by th emethod "catalogue", the market storage is empty, and the file has not been created yet.
             print("Il magazzino è vuoto.\n")
             return None

        else:

            #if error is raised by the method "profit", the market has not sold anything yet. The file 'vendite.csv' is empty.

            print("Il file vendite non è presente. Il negozio non ha ancora venduto nulla.\n")
            return None



  def catalogue(self):
      """
      This method enumerates the content of the 'magazzino.csv' file, describing goods present in the market storage.
      """
      rows=self.read_file("magazzino.csv","catalogue")
      width=20  #width serve a formattare l'otput in colonne equispaziate e ben allineate
      if rows!=None:
         print(f"{self.header[0]:<{width}} \t {self.header[1]:<{width}} \t {self.header[2]:<{width}}\t")
         for line in rows:
             print(f"{line[0]:<{width}} \t {line[1]:<{width}} \t €{line[2]:<{width}}\t")
         print() #aggiunge un 'a capo' al termine dell'elenco




  def add_in_market(self):
    '''
    The method adds goods to the market storage, or updates the already present adding the quantity given by the user.
    To this end, it reads the file 'magazzino.csv', finally printing an informative message to the user.
    '''

    rows=self.read_file("magazzino.csv","add")


    while True:
       name=input("Nome del prodotto: ")
       if all(word.isalpha() for word in name.split())==False:
          print("Nome non valido.")  #check of the product name validity
       else:
          break

    while True:
       try:
          quantity=int(input("Quantità: "))
          if quantity<=0:
            print("Quantità non valida, deve essere positiva.")
          else:
            break
       except ValueError: #if the int cast is not successful, a ValueError is raised while remaining in the loop
        print("Errore: quantità non valida (deve essere un numero intero).")

    update=False  #boolean variable to deal with the case of already present goods and only the available quantity has to be updated.
    for row in rows:
      #reading of the file 'magazzino.csv' to check if the goods are already present, then update them
           if row[0].lower()==name.lower():
               row[1]=str(int(row[1])+quantity)
               update=True

    if update==False:
      #if the goods are not present, a new file row is created
           while True:
               try:
                   buying_price=float(input("Prezzo di acquisto: "))
                   sale_price=float(input("Prezzo di vendita: "))
                   if buying_price<=0 or sale_price<=0:
                      print("Prezzi non validi, devono essere maggiori di zero.")
                   else:
                      break
               except ValueError:
                   print("Errore: prezzi non validi (devono essere numerici).")

           product=[name.lower(),quantity,sale_price,buying_price]
           with open("magazzino.csv","a+", newline='') as file_csv:
              file_writer=csv.writer(file_csv)
              file_writer.writerow(product)

    if update==True:
      #if goods are present, the whole file is written from scratch, updating the specific goods quantity
          with open("magazzino.csv", "w+", newline='') as file_csv:
             file_writer=csv.writer(file_csv)
             file_csv.seek(0)
             file_writer.writerow(self.header)
             for row in rows:
                file_writer.writerow(row)
    # a message of good result in registration or update of goods is printed.
    print(f"AGGIUNTO: {quantity} X {name.lower()}\n")






  def sell(self):
    '''
    The method menages multiple goods selling. if goods are not present in storage, it raises an error, if goods are present before selling,
    it ensures that th edesired uantity is available. It fnally updates the stored quantity of goods after selling.
    Selling activities are stored on the file 'vendite.csv'.
    '''

    shopping_list=[]    #shopping list, empty at the beginning
    condition=True      # condition to deal with multiple selling



    while condition:
          #file 'magazzino.csc' is read.
          rows=self.read_file("magazzino.csv","sell")
          no_quantity=False  # no_quantity menages the case of poor goods quantity

          if rows==None:
            return

          else:
              name_column=[]     #list with all present in storage goods names
              for row in rows:
                 name_column.append(row[0].lower())


              while True:
                 name=input("Nome del prodotto: ")
                 #check on the desidered product name
                 if all(word.isalpha() for word in name.split())==False:
                     print("Nome non valido")
                 else:
                     break

              if name.lower() not in name_column and len(shopping_list)==0:
                  print("Prodotto non presente in magazzino.")
                  return  #if desidered product is not present in storage and the shopping list is empty, the method ends.

              if name.lower() not in name_column and len(shopping_list)!=0:
                #if the shopping list is not empty but the user now wants a missing product, the method ends with selling the already present goods.
                  print("Prodotto non presente in magazzino. Si procede all'acquisto del(dei) prodotto/i già presente/i nella lista della spesa.")
                  break

              while True: #input desidere quantity
                try:
                   quantity_client=int(input("Quantità: "))
                   if quantity_client<=0:
                      print("Quantità non valida, deve essere positiva.")
                   else:
                      break
                except ValueError:
                   print("Errore: quantità non valida (deve essere un numero intero).")  #check on desidered quantity


              with open("magazzino.csv", "w+", newline='') as file_csv:
                 file_writer=csv.writer(file_csv)
                 file_csv.seek(0)
                 file_writer.writerow(self.header)
                 for row in rows:
                     if row[0].lower()==name.lower():

                          if quantity_client > int(row[1]):   #check for the presence of the desidered quantity of goods
                              print("Quantità non disponibile, siamo spiacenti.")
                              no_quantity=True

                          else:
                              row[1]=str(int(row[1])-quantity_client)
                              sell_price=row[2]
                              buy_price=row[3]
                              if row[1]=="0":
                                  #if a product reaches zero available quantity, it is not written on storage file.
                                  continue

                     file_writer.writerow(row)

                 #deal with the case when the desidered quantity of goods is not present
                 if no_quantity==True and len(shopping_list)!=0:
                      print("Si procede ad acquistare i prodotti già in lista.")
                      break    #if the shopping list is not empty, the method sells them only
                 if no_quantity==True and len(shopping_list)==0:
                      return   #if desidered quantity is not present and shopping list is empty, the method ends.

             #if the goods are present and in sufficient amount, the sell is added to the selling list.
              shopping_item=[name.lower(),quantity_client,sell_price,buy_price]
              shopping_list.append(shopping_item)

              correct_choice=False
              while correct_choice==False:
                 #"choice" menages multiple selling
                  choice=input("Aggiungere un altro prodotto? (sì/no): ")
                  if choice.lower() in ["sì","no"]:
                     correct_choice=True
                     if choice.lower()=="no":
                        condition=False
                  else:
                     print("Risposta non valida. Digitare sì o no.")



    total_price=0
    print("VENDITA REGISTRATA")

    #sell is added to the file 'vendite.csv'
    with open("vendite.csv", "a+", newline="") as sell_csv:
           file_writer=csv.writer(sell_csv)
           if sell_csv.tell()==0:
              #tell() tells is the 'vendite.csv' file is empty, thus we are at the first sell. In this case the header is written.
              file_writer.writerow(self.header)

           for item in shopping_list:
               file_writer.writerow(item)
               total_price+=(float(item[1])*float(item[2]))
               print(f"- {item[1]} X {item[0]}: €{float(item[2]):.2f}")

    print(f"Totale: €{total_price:.2f}\n")






  def profit(self):
    '''
    This method analyzes gross and net profits, reading the 'vendite.csv' file.
    '''
    rows=self.read_file("vendite.csv","profit")
    if rows!=None:
       net_profit=0  #si inizializzano a 0 il lordo (gross) e il netto (net).
       gross_profit=0
       net_profit += sum((int(row[1])*(float(row[2])-float(row[3]))) for row in rows)
       gross_profit +=  sum((int(row[1])*(float(row[2]))) for row in rows)
       print(f"Profitto: lordo=€{gross_profit:.2f} netto=€{net_profit:.2f}\n")




  def menu(self):
    """
    This method meanges the interaction user-code, showing a menu with available commands.
    -aggiungi (invoke add_in_market)
    -vendita (invoke metodo sell)
    -profitti (invoke metodo profit)
    -elenca (invoke metodo catalogue)
    -aiuto (enumerates the commands)
    -chiudi (exit the porgram)
    """

    choice=True   #mantains open the command window until the user inserts exit.

    while choice:
       comand=input(("Inserisci un comando: "))
       if comand.lower()=="aiuto":
             print("I comandi disponibili sono i seguenti:\n"+
                    "aggiungi: aggiungi un prodotto al magazzino\n"+
                    "elenca: elenca i prodotti in magazzino\n"+
                    "vendita: registra una vendita effettuata\n"+
                    "profitti: mostra i profitti totali\n"+
                    "aiuto: mostra i possibili comandi\n"+
                    "chiudi: esci dal programma\n")

       elif comand.lower()=="aggiungi":
           self.add_in_market()

       elif comand.lower()=="elenca":
           self.catalogue()

       elif comand.lower()=="vendita":
           self.sell()

       elif comand.lower()=="profitti":
           self.profit()

       elif comand.lower()=="chiudi":
           print("Bye bye\n")
           return

       else:
           print("Comando non valido")  #if the user insets a wrong command, the right ones are reported
           print("I comandi disponibili sono i seguenti:\n"+
                    "aggiungi: aggiungi un prodotto al magazzino\n"+
                    "elenca: elenca i prodotti in magazzino\n"+
                    "vendita: registra una vendita effettuata\n"+
                    "profitti: mostra i profitti totali\n"+
                    "aiuto: mostra i possibili comandi\n"+
                    "chiudi: esci dal programma\n")



################################### MAIN ###########################################

negozio=Market()  #instantiation of the market object
negozio.menu()    #'menu' method call, to start interaction with market
